**The primary purpose of this notebook is to test the OpenEphys sorting "pipeline" with a neuropixel file.

In [1]:
import spikeinterface.widgets as sw
from AODR_session_sorters import OpenEphysSessionSorter as OES
from pyramid import cli
import pandas as pd
import numpy as np
import sys, os
%matplotlib ipympl

Below, we define the relevant paths. You should change these to match the corresponding paths on your machine. If you encounter issues while loading/sorting the file, chances are that your computer is having a hard time accessing the files from cloud storage. If that is the case, you need to transfer your files to a local directory, and also save the files to a local directory.

In [2]:
# Experiment directory top level - should contain the subfolders like ecodes, python, matlab.
expDir = "C:/Users/lt711/Documents/GitHub/Lab_Pipelines/experiments/aodr"
os.chdir(expDir) # regular python files will run from the first open folder in your tree, but notebooks will work from wherever the notebook is stored.
# Directory where the raw files are stored to be converted/sorted
dataSearchPath = "C:/Users/lt711/Box/NP Spike sorting/250224/spikeglx_data/"
# Where the rules for ecodes are stored
pyramidSearchPath = expDir+"/ecodes"
# Conversion specifications
convertSpecs = expDir+"/AODR_experiment_LC.yaml"
# Base directory to save the output files from pyramid (hdf5 files)
baseSaveDir = "C:/Users/lt711/Documents/Kilosort_Neuropixel_Test/" # test output folder
# The session directory (technically not a file, but a folder)
currentFile = "rec10_g4_imec0/"
# Directory to save the output files from sorting
sorted_out = baseSaveDir

sys.path.append(expDir+"/python") # to make sure pyramid can access the custom collectors/enhancers/functions?

Below, we run the sorter/analyzer. With ~4 channels of neural data and a 90 min recording, this should take between 10-20 minutes. The step_names variable defines the sequential processing steps that are defined as methods in AODR_session_sorters OpenEphysSessionSorter class. If you use the step "open_sigui", a gui window will open that allows to interact with the sorting results. Close the gui to continue to the next step and save the output.

In [3]:
sorter = OES(session_dir=dataSearchPath+currentFile,
             out_folder=sorted_out,
             stream_name = 'spikeglx',
             sorter_name='kilosort4',       # If not using kilosort, this is the actual name of the sorter given to spikeinterface. In this case it controls how the probe and temporary binary data are stored
             step_names=[                           # A list of all the sequential steps to take (class methods)
                 'read_data'])                       # Has to be included to do anything else
                 #'run_kilosort4_gui'])               # Run the kilosort4 GUI which will output a phy folder with the results

read_data ...OK


In [4]:
sorter.open_phy(alt_path="C:/Users/lt711/Box/NP Spike sorting/250224/spikeglx_data/rec10_g4_imec0/kilosort4/")

TODO: Now, run Pyramid on the corresponding output to create a TrialFile. Make sure that the phy_reader.params_file points to the appropriate directory based on those you have defined above.

In [ ]:
cli.main(["convert", 
        "--trial-file", trialFileOutputName, 
        "--search-path", pyramidSearchPath, 
        "--experiment", convertSpecs, 
        "--readers", 
        "ttl_reader.session_dir="+dataSearchPath+currentFile,
        "message_reader.session_dir="+dataSearchPath+currentFile,
        "gaze_x_reader.session_dir="+dataSearchPath+currentFile,
        "gaze_y_reader.session_dir="+dataSearchPath+currentFile,
        "pupil_reader.session_dir="+dataSearchPath+currentFile,
        "phy_reader.params_file="+sorted_out+"params.py"])